In [18]:
import torch
import torch.nn as nn
import scipy.io
import torch.nn.functional as F
from torch.autograd import Variable

In [19]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [20]:
dd=torch.randn(10,25,32,155) 
dd=dd.cuda()

In [21]:
class CNN_1(nn.Module):
    def __init__(self, hidden_size, dropout_rate=0.5):
        super(CNN_1, self).__init__()
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
                
        self.conv1_1 = nn.Conv1d(1, 8, kernel_size=8, stride=2, padding=3) 
        self.conv1_2 = nn.Conv1d(1, 8, kernel_size=16, stride=2, padding=7) 
        self.conv1_3 = nn.Conv1d(1, 8, kernel_size=32, stride=2, padding=15) 
        self.conv1_4 = nn.Conv1d(1, 8, kernel_size=64, stride=2, padding=31)
        
        self.norm1 = nn.BatchNorm1d(32) # 
        self.pool1 = nn.MaxPool1d(6, stride=4, padding=1, return_indices=True) # 
        
        self.conv2_1 = nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=1) 
        self.conv2_2 = nn.Conv1d(32, 16, kernel_size=5, stride=1, padding=2) 
        
        self.norm2 = nn.BatchNorm1d(32)
        self.pool2 = nn.MaxPool1d(3, stride=2, padding=1, return_indices=True) # 


        self.conv3_1 = nn.Conv1d(32, 32, kernel_size=3, stride=1, padding=1) 
        self.conv3_2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)         
        
        self.norm3 = nn.BatchNorm1d(64)
        self.pool3 = nn.MaxPool1d(3, stride=2, padding=1, return_indices=True) # 

        self.fc = nn.Linear(5*64, hidden_size)

    def forward(self, x):
        # batch x 1 x raw_feature
                
        encoded = torch.cat([self.conv1_1(x),self.conv1_2(x),self.conv1_3(x),self.conv1_4(x)],1)
        encoded = self.relu(self.norm1(encoded))
        encoded, indices1 = self.pool1(encoded)
        
        encoded = torch.cat([self.conv2_1(encoded),self.conv2_2(encoded)],1)
        encoded = self.relu(self.norm2(encoded))
        encoded, indices2 = self.pool2(encoded)
        
        encoded = torch.cat([self.conv3_1(encoded),self.conv3_2(encoded)],1)
        encoded = self.relu(self.norm3(encoded))
        encoded, indices3 = self.pool3(encoded)
                
        encoded = self.relu(self.fc(encoded.view(encoded.size(0),-1))) # batch x hidden_size
        return encoded

In [22]:
class CNN_0(nn.Module):
    def __init__(self, hidden_size, dropout_rate=0.5):
        super(CNN_0, self).__init__()
        self.hidden_size = hidden_size
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=5, padding=2),
#            nn.init.xavier_uniform(self.conv1.weight),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2))
        self.layer2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2))
        self.dropout = nn.Dropout(dropout_rate)
        
        self.relu = nn.ReLU()
        
        
        self.conv1_1 = nn.Conv2d(1, 8, kernel_size=(1,8), stride=(1,2), padding=(0,3)) 
        self.conv1_2 = nn.Conv2d(1, 8, kernel_size=(1,16), stride=(1,2), padding=(0,7))
        self.conv1_3 = nn.Conv2d(1, 8, kernel_size=(1,32), stride=(1,2), padding=(0,15)) 
        self.conv1_4 = nn.Conv2d(1, 8, kernel_size=(1,64), stride=(1,2), padding=(0,31))
        
        self.norm1 = nn.BatchNorm2d(32) # 
        self.pool1 = nn.AvgPool2d((1,6), stride=(1,4), padding=(0,1)) # 
        
        self.conv2_1 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1) 
        self.conv2_2 = nn.Conv2d(32, 16, kernel_size=5, stride=1, padding=2) 
        
        self.norm2 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d((1,3), stride=(1,2), padding=(0,1)) # 


        self.conv3_1 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1) 
        self.conv3_2 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2)         
        
        self.norm3 = nn.BatchNorm2d(64)
        self.pool3 = nn.AvgPool2d((23,3), stride=(23,2), padding=(0,1)) #      
        
        self.fc = nn.Linear(5*64, hidden_size) #
        

    def forward(self, x):
        # shape: batch x 1 x channel5 x raw_feature1950
        
        encoded = torch.cat([self.conv1_1(x),self.conv1_2(x),self.conv1_3(x),self.conv1_4(x)],1)
        encoded = self.relu(self.norm1(encoded))
        encoded = self.pool1(encoded)
        
        encoded = torch.cat([self.conv2_1(encoded),self.conv2_2(encoded)],1)
        encoded = self.relu(self.norm2(encoded))
        encoded = self.pool2(encoded)
        
        encoded = torch.cat([self.conv3_1(encoded),self.conv3_2(encoded)],1)
        encoded = self.relu(self.norm3(encoded))
        encoded = self.pool3(encoded)
        
        encoded = self.relu(self.fc(encoded.view(encoded.size(0),-1))) # batch x hidden_size
        return encoded  

In [38]:
class Net(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, num_layers, num_classes, num_channel, dropout_rate=0.5):
        super(Net, self).__init__()
        self.input_size = input_size
        self.num_channel = num_channel
        self.num_classes = num_classes
        
        self.hidden1_size = hidden1_size 
        self.hidden2_size = hidden2_size 
        
        self.net0 = CNN_0(hidden1_size)
        self.net1 = CNN_1(hidden1_size)
       
        self.rg = nn.Linear(hidden1_size,1)
        self.rc = nn.Linear(hidden1_size,1)
        self.attention_layer = nn.Linear(self.hidden1_size,1)
        
        self.fc1 = nn.Linear(hidden1_size*2, hidden2_size)
        self.fc2 = nn.Linear(hidden2_size, num_classes)
        self.dropout = nn.Dropout(dropout_rate)
        
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # shape: batch x seq x channel x raw_feature
        x = x.view(x.size(0)*x.size(1),x.size(2),x.size(3)) # (batch*seq) x channel x raw_feature
        
        out1=to_var(torch.zeros((x.size(0), self.num_channel, self.hidden1_size)))
        scores = to_var(torch.zeros((x.size(0), self.num_channel)))
        
        out0 = self.net0(x.view(x.size(0),-1,x.size(1),x.size(2)))# [batch_size*seq, hidden1_size]
        
        for i in range(self.num_channel):
            x_temp = x[:,i,:].contiguous()
            x_temp = x_temp.view(x_temp.size(0),1,x_temp.size(1))
            x_temp = self.net1(x_temp)
            out1[:,i,:] = x_temp.view(x_temp.size(0), -1) # [batch_size*seq x num_channel x hidden1_size]
        
        for i in range(self.num_channel):
            scores[:,i] = self.energy(out0,out1[:,i,:])
        
        scores = F.softmax(scores) # [batch_size*seq, num_channel]
        scores = scores.view(scores.size(0),scores.size(1),-1)
        scores = scores.expand(scores.size(0),scores.size(1),self.hidden1_size) # [batch_size*seq, num_channel, hidden1_size]
        

        out2 = out1 * scores
        out2 = out2.sum(1) # [batch_size*seq, hidden1_size]
        
        out = torch.cat([out0,out2],1) # [batch_size*seq, hidden1_size*2]
        out = self.dropout(out)  # [batch_size*seq, hidden1_size*2]
        out = self.fc1(out) # [batch_size*seq, hidden2_size]
        out = self.relu(out)
        out = self.fc2(out)
        return out
        
    def energy(self, hidden_i, pre_hidden_i): # batch x hidden_size
                
        rate = self.sigmoid(self.rg(hidden_i) + self.rc(pre_hidden_i))
            
        rate = rate.view(rate.size(0),-1)
        rate = rate.expand(rate.size(0),self.hidden1_size)
        out_temp = (1-rate) * hidden_i + rate * pre_hidden_i 
            
        energies = self.attention_layer(out_temp)
        energies= energies.view(energies.size(0)) # new added due to some reasons
        
        return energies

In [39]:
djgnet=Net(input_size=155, hidden1_size=128, hidden2_size=128, num_layers=2, num_classes=2, num_channel=23)

In [40]:
djgnet=djgnet.to('cuda')
yy=djgnet(dd)
print(type(yy))
print(len(yy))
print(yy.shape)
# print(yy[1].shape)

c:\users\djg\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: CUDA out of memory. Tried to allocate 1024.00 KiB (GPU 0; 4.00 GiB total capacity; 2.67 GiB already allocated; 947.20 KiB free; 267.35 MiB cached)